### Import Knet & ONNX

In [37]:
using Pkg;
Pkg.add("Knet")
Pkg.add("IterTools")

  Resolving package versions...
   Updating `~/kafamagore/KnetOnnx.jl/Project.toml`
 [no changes]
   Updating `~/kafamagore/KnetOnnx.jl/Manifest.toml`
 [no changes]
  Resolving package versions...
  Installed IterTools ─ v1.3.0
   Updating `~/kafamagore/KnetOnnx.jl/Project.toml`
  [c8e1da08] + IterTools v1.3.0
   Updating `~/kafamagore/KnetOnnx.jl/Manifest.toml`
  [c8e1da08] + IterTools v1.3.0


In [38]:
using Knet
using IterTools: ncycle, takenth

┌ Info: Precompiling IterTools [c8e1da08-722c-5040-9ed9-7db0dc04731e]
└ @ Base loading.jl:1260


In [10]:
KnetOnnx_PATH = "/Users/syny/kafamagore/KnetOnnx.jl/src"
push!(LOAD_PATH, KnetOnnx_PATH)
using KnetOnnx

┌ Info: Precompiling KnetOnnx [5598a57d-f0b8-44c7-a305-14fa4de76fa0]
└ @ Base loading.jl:1260
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

┌ Warning: Deprecated syntax `(;)` at /Users/syny/.julia/packages/DataFlow/nNf8v/src/syntax/dump.jl:10.
│ Use `begin end` instead.
└ @ ~/.julia/packages/DataFlow/nNf8v/src/syntax/dump.jl:10
┌ Warning: Deprecated syntax `(;)` at /Users/syny/.julia/packages/DataFlow/nNf8v/src/syntax/sugar.jl:96.
│ Use `begin end` instead.
└ @ ~/.julia/packages/DataFlow/nNf8v/src/syntax/sugar.jl:96
┌ Warning: Deprecated syntax `(;)` at /Users/syny/.julia/packages/DataFlow/nNf8v/src/syntax/sugar.jl:107.
│ Use `begin end` instead.
└ @ ~/.julia/packages/DataFlow/nNf8v/src/syntax/sugar.jl:107
┌ Warning: Deprecated syntax `(;)` at /Users/syny/.julia/packages/DataFlow/nNf8v/src/syntax/sugar.jl:255.
│ Use `begin end` instead.
└ @ ~/.julia/packages/DataFlow/nNf8v/src/syntax/sugar.j

### Import MNIST Dataset

In [17]:
include(Knet.dir("data","mnist.jl"))
dtrn,dtst = mnistdata(xsize=(784,:));

┌ Warning: Pkg.installed() is deprecated
└ @ Pkg /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.4/Pkg/src/Pkg.jl:531
┌ Info: Loading MNIST...
└ @ Main /Users/syny/.julia/packages/Knet/bTNMd/data/mnist.jl:33


### Read ONNX Graph

In [82]:
file_path = "./mlp.onnx"
graph = ONNXtoGraph(file_path);

In [83]:
PrintGraph(graph)

model inputs: ["input.1"]
model outputs: ["7"]
(op1) Gemm
	input1: input.1
	input2: fc1.weight
	input3: fc1.bias
	output1: 5
(op2) Relu
	input1: 5
	output1: 6
(op3) Gemm
	input1: 6
	input2: fc2.weight
	input3: fc2.bias
	output1: 7


### Re-construct model in Knet

In [75]:
model = KnetModel(file_path);

In [77]:
x,y = first(dtrn)
model(x)

10×100 Array{Float64,2}:
  0.0494241  -0.0486209  -0.0406266   …   0.0420663   -0.0281929  -0.0272076
 -0.0325614   0.0365905   0.151828        0.00741287   0.0283605   0.0401607
 -0.0404023  -0.135543   -0.0618576      -0.182312     0.0016608   0.0220724
  0.0863582   0.08625     0.0103846       0.0717666    0.0121111   0.0393399
 -0.120319   -0.156512   -0.0768258      -0.166433    -0.199655   -0.145425
  0.0841243  -0.0170137  -0.00637636  …   0.0946109    0.115717    0.0252284
 -0.0702671   0.0153379   0.0127471      -0.167151    -0.133465   -0.0814034
  0.0159794   0.229116    0.0955298       0.16953      0.105005    0.0797005
 -0.110644   -0.064558    0.0337786      -0.0179999   -0.0311736  -0.0758149
  0.0970354   0.0595301  -0.0955818      -0.0161229    0.0820342  -0.0181471

### Re-train the model in Knet

In [86]:
EPOCH = 2
@show accuracy(model, dtst)
progress!(sgd(model, ncycle(dtrn, EPOCH)))
@show accuracy(model, dtst)

accuracy(model, dtst) = 0.088

Stacktrace:
 [1] getindex(::Array{String,1}, ::Int64) at ./array.jl:788
 [2] (::KnetModel)(::Array{Float32,2}, ::Vararg{Any,N} where N) at /Users/syny/kafamagore/KnetOnnx.jl/src/KnetModel.jl:121
 [3] (::Knet.var"#693#694"{Knet.Minimize{IterTools.NCycle{Knet.Data{Tuple{Array{Float32,2},Array{UInt8,1}}}}},Tuple{Array{Float32,2},Array{UInt8,1}}})() at /Users/syny/.julia/packages/AutoGrad/6QsMu/src/core.jl:205
 [4] differentiate(::Function; o::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at /Users/syny/.julia/packages/AutoGrad/6QsMu/src/core.jl:144
 [5] differentiate at /Users/syny/.julia/packages/AutoGrad/6QsMu/src/core.jl:135 [inlined]
 [6] iterate at /Users/syny/.julia/packages/Knet/bTNMd/src/train.jl:23 [inlined]
 [7] iterate at /Users/syny/.julia/packages/Knet/bTNMd/src/progress.jl:69 [inlined]
 [8] progress!(::Knet.Minimize{IterTools.NCycle{Knet.Data{Tuple{Array{Float32,2},Array{UInt8,1}}}}}; o::Base.Iterators.Pairs{Union{},Unio

BoundsError: BoundsError: attempt to access 1-element Array{String,1} at index [2]

In [30]:
predictMNIST(model, dtrn)

UndefVarError: UndefVarError: predictMNIST not defined

In [ ]:
using Images, Plots
function predictMNIST(model, data)
    i = rand(1:100)
    x = rand(data)[1][:,i]
    out = argmax(model(x)); if out==10; out = 0; end
    plot(heatmap(permutedims(reshape(x, 28, 28))), yflip = true, title = ("Model Output: " * string(out)))
end